<a href="https://colab.research.google.com/github/theresaxx/Financial-Disclosure-NLP-10K/blob/main/2_review_dataextraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook reviews the extracted text sections and further processes the results for a improved data quality

Section 7 is splitted to Item 7 and 7A. 

Filters of min and max count of words are applied. 

The different dataframes are merged

### Notebook set up

In [ ]:
import pandas as pd
import numpy as np
import re
import csv

In [ ]:
#Declare whether you are on colab or local
colab = True

In [ ]:
if colab==True:
  
  #mount drive
  from google.colab import drive
  drive.mount('/content/drive')


  #set path to folder in Google Drive
  data_path = "/content/drive/MyDrive/Masterthesis/"

Mounted at /content/drive


### Item 1A

#### Merge and save df



In [ ]:
#Load dataframe

#parsed item 1A
df_2 = pd.read_csv(data_path+"data/item1_parsed/parsed1000.csv", index_col=0)
df_3 = pd.read_csv(data_path+"data/item1_parsed/parsed2000.csv", index_col=0)
df_4 = pd.read_csv(data_path+"data/item1_parsed/parsed3000.csv", index_col=0)
df_5 = pd.read_csv(data_path+"data/item1_parsed/parsed4000.csv", index_col=0)
df_6 = pd.read_csv(data_path+"data/item1_parsed/parsed_tail.csv", index_col=0)

#concat to one df
df_item1_merged = pd.concat([df_2, df_3, df_4, df_5, df_6], axis=0)

#save data
path = data_path+'data/item1_parsed/item1_merged.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_item1_merged.to_csv(f)

#### Load item 1A df 


In [ ]:
#load dataframe
df =  pd.read_csv(data_path+'data/item1_parsed/item1_merged.csv', index_col=0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5964 entries, 5676 to 996
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Symbol        5964 non-null   object
 1   url           5964 non-null   object
 2   date_filed    5964 non-null   object
 3   label         5964 non-null   object
 4   risk_factors  5964 non-null   object
dtypes: object(5)
memory usage: 279.6+ KB


In [ ]:
#remove items which the loop did not find
df_dropped = df[df["risk_factors"].str.len()>50]   #filter for a minimum of 50 characters

In [ ]:
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5689 entries, 5676 to 996
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Symbol        5689 non-null   object
 1   url           5689 non-null   object
 2   date_filed    5689 non-null   object
 3   label         5689 non-null   object
 4   risk_factors  5689 non-null   object
dtypes: object(5)
memory usage: 266.7+ KB


In [ ]:
df_dropped.head(3)

,Symbol,url,date_filed,label,risk_factors
5676,'A',https://www.sec.gov/Archives/edgar/data/109087...,2009-12-21,performance,"Item 1A. Risk Factors Risks, Uncertainties and..."
5236,'A',https://www.sec.gov/Archives/edgar/data/109087...,2010-12-20,performance,"Item 1A. Risk Factors Risks, Uncertainties and..."
4793,'A',https://www.sec.gov/Archives/edgar/data/109087...,2011-12-16,performance,"Item 1A. Risk Factors Risks, Uncertainties and..."


In [ ]:
df_dropped.tail(3)

,Symbol,url,date_filed,label,risk_factors
1982,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2019-02-14,performance,Item 1A. Risk Factors— Risks related to operat...
1491,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2020-02-13,performance,Item 1A. Risk Factors 15Item 1B. Unresolved St...
996,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2021-02-16,performance,Item 1A. Risk Factors— Risks related to operat...


#### Check parsed text

In [ ]:
df_rf = df_dropped
df_rf.head(3)

,Symbol,url,date_filed,label,risk_factors
5676,'A',https://www.sec.gov/Archives/edgar/data/109087...,2009-12-21,performance,"Item 1A. Risk Factors Risks, Uncertainties and..."
5236,'A',https://www.sec.gov/Archives/edgar/data/109087...,2010-12-20,performance,"Item 1A. Risk Factors Risks, Uncertainties and..."
4793,'A',https://www.sec.gov/Archives/edgar/data/109087...,2011-12-16,performance,"Item 1A. Risk Factors Risks, Uncertainties and..."


All extracted text conain "Item 1A Risk Factors" in the beginning, but we need to check if that heading is still contained in the text section, if true, take the last occurence for the secion start.

In [ ]:
#Note: This loop seems complicated, but it considers, that as soon as we shorten the text in the start, we change the position count for the method .search()

def Item1_Search(column:pd.Series):

  rf_list = []

  for rf_text in column:
    text = rf_text[:-1500]     #remove the end, in case the section has a "CAUTIONARY STATEMENT" e.g. https://www.sec.gov/ix?doc=/Archives/edgar/data/1551152/000155115221000008/abbv-20201231.htm#i93d05d2c0e474630bb4c1d6d48ed4446_16
    item1a_regex = re.compile(r".{10}(Item.{1,7})?\b1A\b.{1,5}Risk.{1,5}Factors",  re. IGNORECASE | re.DOTALL)  #.{10} match the first characters to include the first item ocurrence
    start_match = item1a_regex.search(text) # search if the pattern occures another time in the section
    #print(start_match)

    try:
      if start_match != "":       #check if start match contains a match
        start_pos = start_match.start()
        rf_1 = rf_text[start_pos:]
        rf_1_1 = rf_1[10:]      #remove the first 10 characters from .{10}
        rf_2_2 = rf_1_1             #index will start at 0

        rf_2 = rf_2_2[:-1500]      #remove the end, in case the section has a "CAUTIONARY STATEMENT" 
        #print(rf_2)

        try:
          start_match_2 = item1a_regex.search(rf_2) #search if pattern occures again in the section
          #print(start_match_2)   
          if start_match_2 != "":
            start_pos_2 = start_match_2.start()
            rf_3 = rf_2_2[start_pos_2:]
            rf_4 = rf_3[10:]
            rf_list.append(rf_4)
          
          elif start_match_2 == "":
            rf_list.append(rf_1)

        except: 
          rf_list.append(rf_2_2)      
    
    except: 
      rf_list.append(rf_text)

  return rf_list


#train_df_rf["rf_disclosure"] = rf_list

In [ ]:
#first run
run1 = Item1_Search(df_rf["risk_factors"])
df_rf["rf_disclosure_1"] = run1
df_rf.tail(3)

<ipython-input-48-b34fee80dc6e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rf["rf_disclosure_1"] = run1


,Symbol,url,date_filed,label,risk_factors,rf_disclosure_1
1982,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2019-02-14,performance,Item 1A. Risk Factors— Risks related to operat...,Item 1A. Risk Factors. In addition to the othe...
1491,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2020-02-13,performance,Item 1A. Risk Factors 15Item 1B. Unresolved St...,Item 1A. Risk Factors. In addition to the othe...
996,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2021-02-16,performance,Item 1A. Risk Factors— Risks related to operat...,Item 1A. Risk Factors. In addition to the othe...


In [ ]:
#second run
run2 = Item1_Search(df_rf["rf_disclosure_1"])
df_rf["rf_disclosure_2"] = run2
df_rf.tail(3)

<ipython-input-49-7fac51a8932c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rf["rf_disclosure_2"] = run2


,Symbol,url,date_filed,label,risk_factors,rf_disclosure_1,rf_disclosure_2
1982,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2019-02-14,performance,Item 1A. Risk Factors— Risks related to operat...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...
1491,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2020-02-13,performance,Item 1A. Risk Factors 15Item 1B. Unresolved St...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...
996,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2021-02-16,performance,Item 1A. Risk Factors— Risks related to operat...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...


In [ ]:
#third run
run3 = Item1_Search(df_rf["rf_disclosure_2"])
df_rf["rf_disclosure_3"] = run3
df_rf.tail(3)

<ipython-input-50-f87fb0a5856a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rf["rf_disclosure_3"] = run3


,Symbol,url,date_filed,label,risk_factors,rf_disclosure_1,rf_disclosure_2,rf_disclosure_3
1982,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2019-02-14,performance,Item 1A. Risk Factors— Risks related to operat...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...
1491,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2020-02-13,performance,Item 1A. Risk Factors 15Item 1B. Unresolved St...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...
996,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2021-02-16,performance,Item 1A. Risk Factors— Risks related to operat...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...


In [ ]:
#forth run
run4 = Item1_Search(df_rf["rf_disclosure_3"])
df_rf["rf_disclosure_4"] = run4
df_rf.tail(3)

<ipython-input-51-fbff48d5ffdf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rf["rf_disclosure_4"] = run4


,Symbol,url,date_filed,label,risk_factors,rf_disclosure_1,rf_disclosure_2,rf_disclosure_3,rf_disclosure_4
1982,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2019-02-14,performance,Item 1A. Risk Factors— Risks related to operat...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...
1491,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2020-02-13,performance,Item 1A. Risk Factors 15Item 1B. Unresolved St...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...
996,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2021-02-16,performance,Item 1A. Risk Factors— Risks related to operat...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...


In [ ]:
#fifth run
run5 = Item1_Search(df_rf["rf_disclosure_4"])
df_rf["rf_disclosure_5"] = run5
df_rf.tail(3)

<ipython-input-52-395f03f177df>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rf["rf_disclosure_5"] = run5


,Symbol,url,date_filed,label,risk_factors,rf_disclosure_1,rf_disclosure_2,rf_disclosure_3,rf_disclosure_4,rf_disclosure_5
1982,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2019-02-14,performance,Item 1A. Risk Factors— Risks related to operat...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...
1491,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2020-02-13,performance,Item 1A. Risk Factors 15Item 1B. Unresolved St...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...
996,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2021-02-16,performance,Item 1A. Risk Factors— Risks related to operat...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...


In [ ]:
#sixth run
run6 = Item1_Search(df_rf["rf_disclosure_5"])
df_rf["rf_disclosure_final"] = run6
df_rf.tail(3)

<ipython-input-53-fa3b6ee15061>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rf["rf_disclosure_final"] = run6


,Symbol,url,date_filed,label,risk_factors,rf_disclosure_1,rf_disclosure_2,rf_disclosure_3,rf_disclosure_4,rf_disclosure_5,rf_disclosure_final
1982,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2019-02-14,performance,Item 1A. Risk Factors— Risks related to operat...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...
1491,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2020-02-13,performance,Item 1A. Risk Factors 15Item 1B. Unresolved St...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...
996,'ZTS',https://www.sec.gov/Archives/edgar/data/155528...,2021-02-16,performance,Item 1A. Risk Factors— Risks related to operat...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...,Item 1A. Risk Factors. In addition to the othe...


In [ ]:
#check if regex1 is not in secion anymore, for this dataset it was necessary to run function Item1_Search 6 times

regexintext_list=[]

for rf_text in df_rf["rf_disclosure_final"]:
  text = rf_text[:-1500]     #remove the end, in case the section has a "CAUTIONARY STATEMENT" e.g. https://www.sec.gov/ix?doc=/Archives/edgar/data/1551152/000155115221000008/abbv-20201231.htm#i93d05d2c0e474630bb4c1d6d48ed4446_16
  item1a_regex = re.compile(r".{10}(Item.{1,7})?\b1A\b.{1,5}Risk.{1,5}Factors",  re. IGNORECASE | re.DOTALL)  #.{10} match the first characters to include the first item ocurrence

  try:
      start_match = item1a_regex.search(text) # search if the pattern occures another time in the section
      pos = start_match.start()
      regexintext_list.append(pos)

  except:
    regexintext_list.append("not_in_text")


df_rf["regexintext"] = regexintext_list


<ipython-input-54-a9e404fc31d2>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rf["regexintext"] = regexintext_list


In [ ]:
df_rf.head(2)

,Symbol,url,date_filed,label,risk_factors,rf_disclosure_1,rf_disclosure_2,rf_disclosure_3,rf_disclosure_4,rf_disclosure_5,rf_disclosure_final,regexintext
5676,'A',https://www.sec.gov/Archives/edgar/data/109087...,2009-12-21,performance,"Item 1A. Risk Factors Risks, Uncertainties and...","Item 1A. Risk Factors Risks, Uncertainties and...","Item 1A. Risk Factors Risks, Uncertainties and...","Item 1A. Risk Factors Risks, Uncertainties and...","Item 1A. Risk Factors Risks, Uncertainties and...","Item 1A. Risk Factors Risks, Uncertainties and...","Item 1A. Risk Factors Risks, Uncertainties and...",not_in_text
5236,'A',https://www.sec.gov/Archives/edgar/data/109087...,2010-12-20,performance,"Item 1A. Risk Factors Risks, Uncertainties and...","Item 1A. Risk Factors Risks, Uncertainties and...","Item 1A. Risk Factors Risks, Uncertainties and...","Item 1A. Risk Factors Risks, Uncertainties and...","Item 1A. Risk Factors Risks, Uncertainties and...","Item 1A. Risk Factors Risks, Uncertainties and...","Item 1A. Risk Factors Risks, Uncertainties and...",not_in_text


In [ ]:
df_rf_check = df_rf[df_rf.regexintext == "not_in_text"]
df_rf_check.info()
#after checking the text, here are some exceptions such as "item1a_regex" is contained in the heading of each page of item 1A
#therefore those columns are dropped to avoid incomplete text secions 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5641 entries, 5676 to 996
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Symbol               5641 non-null   object
 1   url                  5641 non-null   object
 2   date_filed           5641 non-null   object
 3   label                5641 non-null   object
 4   risk_factors         5641 non-null   object
 5   rf_disclosure_1      5641 non-null   object
 6   rf_disclosure_2      5641 non-null   object
 7   rf_disclosure_3      5641 non-null   object
 8   rf_disclosure_4      5641 non-null   object
 9   rf_disclosure_5      5641 non-null   object
 10  rf_disclosure_final  5641 non-null   object
 11  regexintext          5641 non-null   object
dtypes: object(12)
memory usage: 572.9+ KB


In [ ]:
#only keep columns of interest
df_rf_final = df_rf_check[["Symbol","url","date_filed","label", "rf_disclosure_final"]] 
df_rf_final.head(1)

,Symbol,url,date_filed,label,rf_disclosure_final
5676,'A',https://www.sec.gov/Archives/edgar/data/109087...,2009-12-21,performance,"Item 1A. Risk Factors Risks, Uncertainties and..."


#### Get meta information of risk factors

In [ ]:
df_rf_final[["rf_disclosure_final"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5641 entries, 5676 to 996
Data columns (total 1 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   rf_disclosure_final  5641 non-null   object
dtypes: object(1)
memory usage: 88.1+ KB


In [ ]:
#count number of words
df_rf_final['rf_length'] = df_rf_final["rf_disclosure_final"].apply(lambda x: len(x.split()))
df_rf_final.head()

<ipython-input-59-523113582a02>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rf_final['rf_length'] = df_rf_final["rf_disclosure_final"].apply(lambda x: len(x.split()))


,Symbol,url,date_filed,label,rf_disclosure_final,rf_length
5676,'A',https://www.sec.gov/Archives/edgar/data/109087...,2009-12-21,performance,"Item 1A. Risk Factors Risks, Uncertainties and...",6248
5236,'A',https://www.sec.gov/Archives/edgar/data/109087...,2010-12-20,performance,"Item 1A. Risk Factors Risks, Uncertainties and...",6426
4793,'A',https://www.sec.gov/Archives/edgar/data/109087...,2011-12-16,performance,"Item 1A. Risk Factors Risks, Uncertainties and...",6369
4343,'A',https://www.sec.gov/Archives/edgar/data/109087...,2012-12-20,performance,"Item 1A. Risk Factors Risks, Uncertainties and...",6610
3887,'A',https://www.sec.gov/Archives/edgar/data/109087...,2013-12-19,performance,"ITEM 1A. RISK FACTORS Risks, Uncertainties and...",7481


In [ ]:
df_rf_final.describe()

,rf_length
count,5641.000000
mean,7862.253324
std,7660.332011
min,78.000000
25%,4170.000000
50%,6977.000000
75%,10211.000000
max,200057.000000


In [ ]:
#text should have at least 1000 words
df_rf_check_min_drop = df_rf_final[df_rf_final.rf_length < 1000]
df_rf_check_min_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173 entries, 1027 to 689
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Symbol               173 non-null    object
 1   url                  173 non-null    object
 2   date_filed           173 non-null    object
 3   label                173 non-null    object
 4   rf_disclosure_final  173 non-null    object
 5   rf_length            173 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 9.5+ KB


In [ ]:
#text should have max of 15.000 words
df_rf_check_max = df_rf_final[df_rf_final.rf_length > 15000]
df_rf_check_max.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 6556 to 996
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Symbol               392 non-null    object
 1   url                  392 non-null    object
 2   date_filed           392 non-null    object
 3   label                392 non-null    object
 4   rf_disclosure_final  392 non-null    object
 5   rf_length            392 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 21.4+ KB


In [ ]:
df_rf_final_3 = df_rf_final[ (df_rf_final ["rf_length"] < 15000 ) & (df_rf_final ["rf_length"] > 1000) ]

In [ ]:
df_rf_final_3.describe()

,rf_length
count,5076.000000
mean,7038.595154
std,3434.083386
min,1001.000000
25%,4252.500000
50%,6778.500000
75%,9539.000000
max,14993.000000


In [ ]:
df_rf_final_3 = df_rf_final_3.drop(columns = ["rf_length"])
df_rf_final_3.head(1)

,Symbol,url,date_filed,label,rf_disclosure_final
5676,'A',https://www.sec.gov/Archives/edgar/data/109087...,2009-12-21,performance,"Item 1A. Risk Factors Risks, Uncertainties and..."


#### Save final df_rf

In [ ]:
#save data
path = data_path+'data/item1_parsed/item1_final_filtered.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_rf_final_3.to_csv(f)

### Item 7 + 7A

#### Merge and save df

In [ ]:
#Load dataframe

#parsed item 7
df_2 = pd.read_csv(data_path+"data/item7_parsed/parsed1000.csv", index_col=0)
df_3 = pd.read_csv(data_path+"data/item7_parsed/parsed2000.csv", index_col=0)
df_4 = pd.read_csv(data_path+"data/item7_parsed/parsed3000.csv", index_col=0)
df_5 = pd.read_csv(data_path+"data/item7_parsed/parsed4000.csv", index_col=0)
df_6 = pd.read_csv(data_path+"data/item7_parsed/parsedtail.csv", index_col=0)

#concat to one df
df_item7_merged = pd.concat([df_2, df_3, df_4, df_5, df_6], axis=0)

#save data
path = data_path+'data/item7_parsed/item7_merged.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_item7_merged.to_csv(f)

#### Load Item 7 + 7A text

In [ ]:
#load dataframe
df_7 =  pd.read_csv(data_path+'data/item7_parsed/item7_merged.csv', index_col=0)

#### Check parsed text

In [ ]:
df_7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5115 entries, 5676 to 1491
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Symbol      5115 non-null   object
 1   url         5115 non-null   object
 2   date_filed  5115 non-null   object
 3   label       5115 non-null   object
 4   section7    5115 non-null   object
 5   mda         5115 non-null   object
 6   qqdmr       5115 non-null   object
dtypes: object(7)
memory usage: 319.7+ KB


In [ ]:
df_7.head()

,Symbol,url,date_filed,label,section7
5676,'A',https://www.sec.gov/Archives/edgar/data/109087...,2009-12-21,performance,Item 7. Management's Discussion and Analysis o...
5236,'A',https://www.sec.gov/Archives/edgar/data/109087...,2010-12-20,performance,Item 7. Management's Discussion and Analysis o...
4793,'A',https://www.sec.gov/Archives/edgar/data/109087...,2011-12-16,performance,Item 7. Management's Discussion and Analysis o...
4343,'A',https://www.sec.gov/Archives/edgar/data/109087...,2012-12-20,performance,Item 7. Management's Discussion and Analysis o...
3887,'A',https://www.sec.gov/Archives/edgar/data/109087...,2013-12-19,performance,Item 7. Management's Discussion and Analysis o...


In [ ]:
#remove items which the loop did not find
df_7_dropped = df_7[df_7["section7"].str.len()>50]   #filter for a minimum of 50 characters

In [ ]:
df_7_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5115 entries, 5676 to 1491
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Symbol      5115 non-null   object
 1   url         5115 non-null   object
 2   date_filed  5115 non-null   object
 3   label       5115 non-null   object
 4   section7    5115 non-null   object
dtypes: object(5)
memory usage: 239.8+ KB


#### Split Item 7 and 7A

In [ ]:
def splitSection(column:pd.Series):

  item7_list = []
  item7A_list = []

  for item_text in column:
    item7A_regex = re.compile(r"(Item.{1,7})?\b7A\b.{1,5}Quantitative.{1,5}and.{1,5}Qualitative",  re. IGNORECASE | re.DOTALL)
    
    start_match = item7A_regex.search(item_text) #search if 7a pattern occurs in the text
    #print(start_match)

    try: 
      if start_match != "":       #check if start match contains a match
        start_pos = start_match.start()
        item7A = item_text[start_pos:-1]  
        #print(item7A) 
        end_pos = start_match.end() 
        #print(end_pos)    
        item7 = item_text[0:end_pos]
        #print(item7)

        item7A_list.append(item7A) 
        item7_list.append(item7)   
    
    except: 
      item7_list.append("not_found")
      item7A_list.append("not_found")

  return item7_list, item7A_list


#df_7_dropped["mda"] = item7_list
#df_7_dropped["qqdmr"] = item7A_list

In [ ]:
mda, qqmdr = splitSection(df_7_dropped["section7"])
df_7_dropped["mda"] = mda
df_7_dropped["qqdmr"] = qqmdr

In [ ]:
df_7_dropped = df_7_dropped[df_7_dropped["mda"].str.len()>50]   #filter for a minimum of 50 characters
df_7 = df_7_dropped[df_7_dropped["qqdmr"].str.len()>50]   #filter for a minimum of 50 characters
df_7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4305 entries, 5676 to 995
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Symbol      4305 non-null   object
 1   url         4305 non-null   object
 2   date_filed  4305 non-null   object
 3   label       4305 non-null   object
 4   section7    4305 non-null   object
 5   mda         4305 non-null   object
 6   qqdmr       4305 non-null   object
dtypes: object(7)
memory usage: 269.1+ KB


#### Get meta information of mda

In [ ]:
df_7[["mda"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4305 entries, 5676 to 995
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   mda     4305 non-null   object
dtypes: object(1)
memory usage: 67.3+ KB


In [ ]:
df_7['mda_length'] = df_7["mda"].apply(lambda x: len(x.split()))
df_7.head()

,Symbol,url,date_filed,label,section7,mda,qqdmr,mda_length
5676,'A',https://www.sec.gov/Archives/edgar/data/109087...,2009-12-21,performance,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,13451
5236,'A',https://www.sec.gov/Archives/edgar/data/109087...,2010-12-20,performance,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,12856
4793,'A',https://www.sec.gov/Archives/edgar/data/109087...,2011-12-16,performance,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,13561
4343,'A',https://www.sec.gov/Archives/edgar/data/109087...,2012-12-20,performance,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,15088
3887,'A',https://www.sec.gov/Archives/edgar/data/109087...,2013-12-19,performance,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,14611


In [ ]:
df_7.describe()

,mda_length
count,4305.000000
mean,13102.791870
std,10564.726227
min,14.000000
25%,8091.000000
50%,11800.000000
75%,16204.000000
max,140370.000000


In [ ]:
#text should have at least 1000 words
df_7_check_min_drop = df_7[df_7.mda_length < 1000]
df_7_check_min_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 1044 to 6225
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Symbol      500 non-null    object
 1   url         500 non-null    object
 2   date_filed  500 non-null    object
 3   label       500 non-null    object
 4   section7    500 non-null    object
 5   mda         500 non-null    object
 6   qqdmr       500 non-null    object
 7   mda_length  500 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 35.2+ KB


In [ ]:
#text should have max of 20.000 words
df_7_check_max = df_7[df_7.mda_length > 20000]
df_7_check_max.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 581 entries, 6139 to 995
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Symbol      581 non-null    object
 1   url         581 non-null    object
 2   date_filed  581 non-null    object
 3   label       581 non-null    object
 4   section7    581 non-null    object
 5   mda         581 non-null    object
 6   qqdmr       581 non-null    object
 7   mda_length  581 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 40.9+ KB


In [ ]:
#drop rows
df_7_final_3 = df_7[ (df_7 ["mda_length"] < 20000 ) & (df_7 ["mda_length"] > 1000) ]
df_7_final_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3224 entries, 5676 to 993
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Symbol      3224 non-null   object
 1   url         3224 non-null   object
 2   date_filed  3224 non-null   object
 3   label       3224 non-null   object
 4   section7    3224 non-null   object
 5   mda         3224 non-null   object
 6   qqdmr       3224 non-null   object
 7   mda_length  3224 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 226.7+ KB


In [ ]:
df_7_final_3.describe()

,mda_length
count,3224.000000
mean,11765.524194
std,3917.539818
min,1008.000000
25%,8999.750000
50%,11675.500000
75%,14552.000000
max,19995.000000


In [ ]:
df_7_final = df_7_final_3.drop(columns = ["mda_length"])
df_7_final.head(1)

,Symbol,url,date_filed,label,section7,mda,qqdmr
5676,'A',https://www.sec.gov/Archives/edgar/data/109087...,2009-12-21,performance,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...


#### Get meta information of qqdmr

In [ ]:
df_7_final[["qqdmr"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3224 entries, 5676 to 993
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   qqdmr   3224 non-null   object
dtypes: object(1)
memory usage: 50.4+ KB


In [ ]:
df_7_final['qqdmr_length'] = df_7_final["qqdmr"].apply(lambda x: len(x.split()))
df_7_final.tail()

,Symbol,url,date_filed,label,section7,mda,qqdmr,qqdmr_length
2934,'ZBRA',https://www.sec.gov/Archives/edgar/data/877212...,2017-02-27,performance,Item 7.Management’s Discussion and Analysis of...,Item 7.Management’s Discussion and Analysis of...,Item 7A. Quantitative and Qualitative Disclosu...,521
2460,'ZBRA',https://www.sec.gov/Archives/edgar/data/877212...,2018-02-22,performance,Item 7.Management’s Discussion and Analysis of...,Item 7.Management’s Discussion and Analysis of...,Item 7A. Quantitative and Qualitative Disclosu...,512
1979,'ZBRA',https://www.sec.gov/Archives/edgar/data/877212...,2019-02-14,performance,Item 7.Management’s Discussion and Analysis of...,Item 7.Management’s Discussion and Analysis of...,Item 7A. Quantitative and Qualitative Disclosu...,442
1488,'ZBRA',https://www.sec.gov/Archives/edgar/data/877212...,2020-02-13,performance,Item 7.Management’s Discussion and Analysis of...,Item 7.Management’s Discussion and Analysis of...,Item 7A. Quantitative and Qualitative Disclosu...,533
993,'ZBRA',https://www.sec.gov/Archives/edgar/data/877212...,2021-02-11,performance,Item 7.Management’s Discussion and Analysis of...,Item 7.Management’s Discussion and Analysis of...,Item 7A. Quantitative and Qualitative Disclosu...,473


In [ ]:
df_7_final.describe()

,qqdmr_length
count,3224.000000
mean,1583.696030
std,3609.115114
min,10.000000
25%,312.000000
50%,690.500000
75%,1115.250000
max,43228.000000


In [ ]:
#text should have at least 1000 words
df_7_check_min_drop = df_7_final[df_7_final.qqdmr_length < 300]
df_7_check_min_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 777 entries, 5676 to 990
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Symbol        777 non-null    object
 1   url           777 non-null    object
 2   date_filed    777 non-null    object
 3   label         777 non-null    object
 4   section7      777 non-null    object
 5   mda           777 non-null    object
 6   qqdmr         777 non-null    object
 7   qqdmr_length  777 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 54.6+ KB


In [ ]:
#drop rows
df_7_final_4 = df_7_final[(df_7_final ["qqdmr_length"] > 300)]
df_7_final_4

,Symbol,url,date_filed,label,section7,mda,qqdmr,qqdmr_length
5236,'A',https://www.sec.gov/Archives/edgar/data/109087...,2010-12-20,performance,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,516
4793,'A',https://www.sec.gov/Archives/edgar/data/109087...,2011-12-16,performance,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,540
4343,'A',https://www.sec.gov/Archives/edgar/data/109087...,2012-12-20,performance,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,426
3887,'A',https://www.sec.gov/Archives/edgar/data/109087...,2013-12-19,performance,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,426
3422,'A',https://www.sec.gov/Archives/edgar/data/109087...,2014-12-22,performance,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,430
...,...,...,...,...,...,...,...,...
2934,'ZBRA',https://www.sec.gov/Archives/edgar/data/877212...,2017-02-27,performance,Item 7.Management’s Discussion and Analysis of...,Item 7.Management’s Discussion and Analysis of...,Item 7A. Quantitative and Qualitative Disclosu...,521
2460,'ZBRA',https://www.sec.gov/Archives/edgar/data/877212...,2018-02-22,performance,Item 7.Management’s Discussion and Analysis of...,Item 7.Management’s Discussion and Analysis of...,Item 7A. Quantitative and Qualitative Disclosu...,512
1979,'ZBRA',https://www.sec.gov/Archives/edgar/data/877212...,2019-02-14,performance,Item 7.Management’s Discussion and Analysis of...,Item 7.Management’s Discussion and Analysis of...,Item 7A. Quantitative and Qualitative Disclosu...,442
1488,'ZBRA',https://www.sec.gov/Archives/edgar/data/877212...,2020-02-13,performance,Item 7.Management’s Discussion and Analysis of...,Item 7.Management’s Discussion and Analysis of...,Item 7A. Quantitative and Qualitative Disclosu...,533


In [ ]:
df_7_final_5 = df_7_final_4.drop(columns = ["qqdmr_length"])
df_7_final_5.head(1)

,Symbol,url,date_filed,label,section7,mda,qqdmr
5236,'A',https://www.sec.gov/Archives/edgar/data/109087...,2010-12-20,performance,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...


#### save final section 7 df

In [ ]:
#save data
path = data_path+'data/item7_parsed/item7_final_filtered.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_7_final_5.to_csv(f)

### Merge dfs, filter df, show label distribution, save final merged df

In [ ]:
df_item1 =  pd.read_csv(data_path+'data/item1_parsed/item1_final_filtered.csv', index_col=0)
df_item7 =  pd.read_csv(data_path+'data/item7_parsed/item7_final_filtered.csv', index_col=0)

In [ ]:
df_item1['index'] = df_item1.index
df_item7['index'] = df_item7.index

In [ ]:
df_item1.head()

,Symbol,url,date_filed,label,rf_disclosure_final,index
5676,'A',https://www.sec.gov/Archives/edgar/data/109087...,2009-12-21,performance,"Item 1A. Risk Factors Risks, Uncertainties and...",5676
5236,'A',https://www.sec.gov/Archives/edgar/data/109087...,2010-12-20,performance,"Item 1A. Risk Factors Risks, Uncertainties and...",5236
4793,'A',https://www.sec.gov/Archives/edgar/data/109087...,2011-12-16,performance,"Item 1A. Risk Factors Risks, Uncertainties and...",4793
4343,'A',https://www.sec.gov/Archives/edgar/data/109087...,2012-12-20,performance,"Item 1A. Risk Factors Risks, Uncertainties and...",4343
3887,'A',https://www.sec.gov/Archives/edgar/data/109087...,2013-12-19,performance,"ITEM 1A. RISK FACTORS Risks, Uncertainties and...",3887


In [ ]:
df3 = df_item1.merge(df_item7, on='index', how='left')

In [ ]:
df3.head()

,Symbol_x,url_x,date_filed_x,label_x,rf_disclosure_final,index,Symbol_y,url_y,date_filed_y,label_y,section7,mda,qqdmr
0,'A',https://www.sec.gov/Archives/edgar/data/109087...,2009-12-21,performance,"Item 1A. Risk Factors Risks, Uncertainties and...",5676,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,'A',https://www.sec.gov/Archives/edgar/data/109087...,2010-12-20,performance,"Item 1A. Risk Factors Risks, Uncertainties and...",5236,'A',https://www.sec.gov/Archives/edgar/data/109087...,2010-12-20,performance,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...
2,'A',https://www.sec.gov/Archives/edgar/data/109087...,2011-12-16,performance,"Item 1A. Risk Factors Risks, Uncertainties and...",4793,'A',https://www.sec.gov/Archives/edgar/data/109087...,2011-12-16,performance,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...
3,'A',https://www.sec.gov/Archives/edgar/data/109087...,2012-12-20,performance,"Item 1A. Risk Factors Risks, Uncertainties and...",4343,'A',https://www.sec.gov/Archives/edgar/data/109087...,2012-12-20,performance,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...
4,'A',https://www.sec.gov/Archives/edgar/data/109087...,2013-12-19,performance,"ITEM 1A. RISK FACTORS Risks, Uncertainties and...",3887,'A',https://www.sec.gov/Archives/edgar/data/109087...,2013-12-19,performance,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...


In [ ]:
df3_3 = df3[300:303]
df3_3

,Symbol_x,url_x,date_filed_x,label_x,rf_disclosure_final,index,Symbol_y,url_y,date_filed_y,label_y,section7,mda,qqdmr
300,'AMAT',https://www.sec.gov/Archives/edgar/data/6951/0...,2011-12-06,performance,Item 1A: Risk Factors The following factors co...,6138,'AMAT',https://www.sec.gov/Archives/edgar/data/6951/0...,2011-12-06,performance,Item 7: Managements Discussion and Analysis o...,Item 7: Managements Discussion and Analysis o...,Item 7A: Quantitative and Qualitative Disclosu...
301,'AMAT',https://www.sec.gov/Archives/edgar/data/6951/0...,2012-12-05,performance,Item 1A:Risk FactorsThe following factors coul...,4371,'AMAT',https://www.sec.gov/Archives/edgar/data/6951/0...,2012-12-05,performance,Item 7:Management’s Discussion and Analysis of...,Item 7:Management’s Discussion and Analysis of...,Item 7A:Quantitative and Qualitative Disclosur...
302,'AMAT',https://www.sec.gov/Archives/edgar/data/6951/0...,2013-12-04,performance,Item 1A:Risk FactorsThe following factors coul...,3916,'AMAT',https://www.sec.gov/Archives/edgar/data/6951/0...,2013-12-04,performance,Item 7:Management’s Discussion and Analysis of...,Item 7:Management’s Discussion and Analysis of...,Item 7A:Quantitative and Qualitative Disclosur...


In [ ]:
#drop nan
df4 = df3.dropna()
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2178 entries, 1 to 5061
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Symbol_x             2178 non-null   object
 1   url_x                2178 non-null   object
 2   date_filed_x         2178 non-null   object
 3   label_x              2178 non-null   object
 4   rf_disclosure_final  2178 non-null   object
 5   index                2178 non-null   int64 
 6   Symbol_y             2178 non-null   object
 7   url_y                2178 non-null   object
 8   date_filed_y         2178 non-null   object
 9   label_y              2178 non-null   object
 10  section7             2178 non-null   object
 11  mda                  2178 non-null   object
 12  qqdmr                2178 non-null   object
dtypes: int64(1), object(12)
memory usage: 238.2+ KB


In [ ]:
df4_4 = df4[350:355]
df4_4

,Symbol_x,url_x,date_filed_x,label_x,rf_disclosure_final,index,Symbol_y,url_y,date_filed_y,label_y,section7,mda,qqdmr
823,'CARR',https://www.sec.gov/Archives/edgar/data/178318...,2021-02-09,performance,ITEM 1A. RISK FACTORS RISK FACTOR SUMMARYRisks...,590,'CARR',https://www.sec.gov/Archives/edgar/data/178318...,2021-02-09,performance,ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,ITEM 7A. QUANTITATIVE AND QUALITATIVE DISCLOSU...
857,'CBOE',https://www.sec.gov/Archives/edgar/data/137431...,2018-02-22,low_performance,Item 1A. Risk Factors. The risks and uncertain...,2073,'CBOE',https://www.sec.gov/Archives/edgar/data/137431...,2018-02-22,low_performance,Item 7. Management’s Discussion and Analysis o...,Item 7. Management’s Discussion and Analysis o...,Item 7A.Quantitative and Qualitative Disclosur...
858,'CBOE',https://www.sec.gov/Archives/edgar/data/137431...,2019-02-22,performance,Item 1A. Risk Factors. The risks and uncertain...,1584,'CBOE',https://www.sec.gov/Archives/edgar/data/137431...,2019-02-22,performance,Item 7. Management’s Discussion and Analysis o...,Item 7. Management’s Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...
859,'CBOE',https://www.sec.gov/Archives/edgar/data/137431...,2020-02-21,low_performance,Item 1A. Risk Factors. The risks and uncertain...,1089,'CBOE',https://www.sec.gov/Archives/edgar/data/137431...,2020-02-21,low_performance,Item 7. Management’s Discussion and Analysis o...,Item 7. Management’s Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...
867,'CBRE',https://www.sec.gov/Archives/edgar/data/113811...,2016-02-29,performance,Item 1A. Risk Factors Set forth below and else...,3028,'CBRE',https://www.sec.gov/Archives/edgar/data/113811...,2016-02-29,performance,Item 7. Managements Discussion and Analysis o...,Item 7. Managements Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...


In [ ]:
#create final dataframes
df_final = df4.drop(columns = ["Symbol_y", "url_y", "date_filed_y", "label_y"])
df_final.head()

,Symbol_x,url_x,date_filed_x,label_x,rf_disclosure_final,index,section7,mda,qqdmr
1,'A',https://www.sec.gov/Archives/edgar/data/109087...,2010-12-20,performance,"Item 1A. Risk Factors Risks, Uncertainties and...",5236,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...
2,'A',https://www.sec.gov/Archives/edgar/data/109087...,2011-12-16,performance,"Item 1A. Risk Factors Risks, Uncertainties and...",4793,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...
3,'A',https://www.sec.gov/Archives/edgar/data/109087...,2012-12-20,performance,"Item 1A. Risk Factors Risks, Uncertainties and...",4343,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...
4,'A',https://www.sec.gov/Archives/edgar/data/109087...,2013-12-19,performance,"ITEM 1A. RISK FACTORS Risks, Uncertainties and...",3887,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...
5,'A',https://www.sec.gov/Archives/edgar/data/109087...,2014-12-22,performance,"ITEM 1A. RISK FACTORS Risks, Uncertainties and...",3422,Item 7. Management's Discussion and Analysis o...,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...


#### label distribution

In [ ]:
df_count_label = df_final[(df_final ["label_x"] ==  "performance")]
df_count_label.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1897 entries, 1 to 5061
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Symbol_x             1897 non-null   object
 1   url_x                1897 non-null   object
 2   date_filed_x         1897 non-null   object
 3   label_x              1897 non-null   object
 4   rf_disclosure_final  1897 non-null   object
 5   index                1897 non-null   int64 
 6   section7             1897 non-null   object
 7   mda                  1897 non-null   object
 8   qqdmr                1897 non-null   object
dtypes: int64(1), object(8)
memory usage: 148.2+ KB


In [ ]:
df_count_label = df_final[(df_final ["label_x"] ==  "low_performance")]
df_count_label.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 281 entries, 52 to 5054
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Symbol_x             281 non-null    object
 1   url_x                281 non-null    object
 2   date_filed_x         281 non-null    object
 3   label_x              281 non-null    object
 4   rf_disclosure_final  281 non-null    object
 5   index                281 non-null    int64 
 6   section7             281 non-null    object
 7   mda                  281 non-null    object
 8   qqdmr                281 non-null    object
dtypes: int64(1), object(8)
memory usage: 22.0+ KB


Because of the label distribution, we adjust the label for training of dataframe, see notebook 4

### Save dataframe 

In [ ]:
#save data
path = data_path+'data/merged/all_merged.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_final.to_csv(f)

#### backlog

In [ ]:
#Note: This loop seems complicated, but we need to consider, as soon as we shorten the text, we change the position count for the method .search()

rf_list = []

for rf_text in train_df_rf["risk_factors"]:
  text = rf_text[:-1500]     #remove the end, in case the section has a "CAUTIONARY STATEMENT" 
  item1a_regex = re.compile(r".{10}(Item.{1,7})?\b1A\b.{1,5}Risk.{1,5}Factors",  re. IGNORECASE | re.DOTALL)  #.{10} match the first characters to include the first item ocurrence
  start_match = item1a_regex.search(text) # search if the pattern occures another time in the section
  #print(start_match)

  try:
    if start_match != "":       #check if start match contains a match
      start_pos = start_match.start()
      rf_1 = rf_text[start_pos:]
      rf_1_1 = rf_1[10:]      #remove the first 10 characters from .{10}
      rf_2_2 = rf_1_1             #index will start at 0

      rf_2 = rf_2_2[:-1500]      #remove the end, in case the section has a "CAUTIONARY STATEMENT" 
      #print(rf_2)

      try:
        start_match_2 = item1a_regex.search(rf_2) #search if pattern occures again in the section
        #print(start_match_2)   
        if start_match_2 != "":
          start_pos_2 = start_match_2.start()
          rf_3 = rf_2_2[start_pos_2:]
          rf_4 = rf_3[10:]
          rf_list.append(rf_4)
        
        elif start_match_2 == "":
          rf_list.append(rf_1)

      except: 
        rf_list.append(rf_2_2)      
  
  except: 
    rf_list.append(rf_text)


train_df_rf["rf_disclosure"] = rf_list

<ipython-input-81-22b2b5438ae2>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_rf["rf_disclosure"] = rf_list


In [ ]:
item1a_regex = re.compile(r".{10}(Item.{1,7})?\b1A\b.{1,5}Risk.{1,5}Factors",  re. IGNORECASE | re.DOTALL)  #include the first item ocurrence
rf_list = []

for rf_text in train_df_rf["risk_factors"]:
  text = rf_text[:-1500]     #remove the end, in case the section has a "CAUTIONARY STATEMENT" 
  section_start_match = item1a_regex.search(text) # search if the pattern occures another time in the section
  print(section_start_match)

  if section_start_match != "":
    section_start_pos = section_start_match.start()
    rf_1 = rf_text[section_start_pos:-1]        #important, take the text from first variable "rf_text", not "text" because we cut the end
    print(rf_1)
    rf_2 = rf_1[10:-1500]          #remove the end, in case the section has a "CAUTIONARY STATEMENT" 
    print(rf_2)    
    section_start_match_2 = item1a_regex.search(rf_2)  
    print(section_start_match_2)

    if section_start_match_2 != "":
      section_start_pos_2 = section_start_match_2.start()
      rf_3 = rf_1[section_start_pos_2:-1]        #important, take the text from "rf_1", not "rf_2" because we cut the end
      rf_list.append(rf_3)
    
    elif section_start_match_2 == "":
      rf_list.append(rf_1)
 
  else: 
    rf_list.append(rf_text)


train_df_rf["rf_disclosure_1"] = rf_list


<re.Match object; span=(9426, 9457), match=' Contents ITEM 1A. RISK FACTORS'>
 Contents ITEM 1A. RISK FACTORSThe following discussion describes certain risk factors that we believe could affect our business and prospects. These risk factors are in addition to those set forth elsewhere in this report.Our results of operations could be adversely impacted by manufacturer pricing changes and fewer generic pharmaceutical launches.In fiscal 2018, we continued to experience an unfavorable brand and generic pharmaceutical pricing environment, which negatively impacted our Pharmaceutical Distribution Services reportable segment profit and our consolidated operating earnings. We expect this to continue in fiscal 2019, which could have a material and adverse effect on our results of operations.Our contractual arrangements with pharmaceutical manufacturers for the purchase of brand pharmaceutical products generally use wholesale acquisition cost ("WAC") as the reference price. Similarly, we sell b

AttributeError: ignored

In [ ]:
item1a_regex = re.compile(r".{10,11}(Item.{1,7})?\b1A\b.{1,5}Risk.{1,5}Factors",  re. IGNORECASE | re.DOTALL)  #include the first item ocurrence
rf_list = []

for rf_text in train_df_rf["risk_factors"]:
  text = rf_text[:-1500]     #remove the end, in case the section has a "CAUTIONARY STATEMENT" 
  start_match = item1a_regex.search(text) # search if the pattern occures another time in the section
  print(start_match)

  if start_match != "":
    start_pos = start_match.start()
    rf_1 = rf_text[start_pos:-1]    
    rf_2 = rf_1[10:-1500]          #remove the end, in case the section has a "CAUTIONARY STATEMENT" 
    print(rf_2)
    start_match_2 = item1a_regex.search(rf_2) #search if pattern occures again in the section

    print(start_match_2)   
    if start_match_2 != "":
      start_pos_2 = start_match_2.start()
      rf_3 = rf_2[start_pos_2:-1]
      rf_4 = rf_3[10:-1]
      rf_list.append(rf_4)
    
    elif start_match_2 == "":
      rf_list.append(rf_1)
    
  else: 
    rf_list.append(text)


train_df_rf["rf_disclosure"] = rf_list

<re.Match object; span=(9425, 9457), match='f Contents ITEM 1A. RISK FACTORS'>
 ITEM 1A. RISK FACTORSThe following discussion describes certain risk factors that we believe could affect our business and prospects. These risk factors are in addition to those set forth elsewhere in this report.Our results of operations could be adversely impacted by manufacturer pricing changes and fewer generic pharmaceutical launches.In fiscal 2018, we continued to experience an unfavorable brand and generic pharmaceutical pricing environment, which negatively impacted our Pharmaceutical Distribution Services reportable segment profit and our consolidated operating earnings. We expect this to continue in fiscal 2019, which could have a material and adverse effect on our results of operations.Our contractual arrangements with pharmaceutical manufacturers for the purchase of brand pharmaceutical products generally use wholesale acquisition cost ("WAC") as the reference price. Similarly, we sell brand pha

AttributeError: ignored

In [ ]:
item1a_regex = re.compile(r".{10,20}(Item.{1,7})?\b1A\b.{1,5}Risk.{1,5}Factors",  re. IGNORECASE | re.DOTALL)  #include the first item ocurrence
rf_list = []

for rf_text in train_df_rf["rf_disclosure_1"]:
  text = rf_text[:-1500]     #remove the end, in case the section has a "CAUTIONARY STATEMENT" 
  section_start_match = item1a_regex.search(text) # search if the pattern occures another time in the section
  #print(section_start_match)

  if section_start_match != "":
    section_start_pos = section_start_match.start()
    rf_1 = rf_text[section_start_pos:-1]   #important, take the text from first variable "rf_text", not "text" because we cut the end
    rf_list.append(rf_1)
 
  else: 
    rf_list.append(text)


train_df_rf["rf_disclosure_2"] = rf_list


<re.Match object; span=(0, 41), match='.7Table of Contents ITEM 1A. RISK FACTORS'>
<re.Match object; span=(0, 41), match='.7Table of Contents ITEM 1A. RISK FACTORS'>


<ipython-input-147-05a08dc91e03>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_rf["rf_disclosure_2"] = rf_list


In [ ]:
#filter for items which the loop did not find
train_df_pe =  train_df_1[(train_df_1['mdaf']=="parsing error") ]
train_df_pes = train_df_pe.shape
print("Rows where a parsing error occured {} ".format(*train_df_pes))

train_df_rf =  train_df_1[ (train_df_1['risk_factors']=="not found") & (train_df_1['mdaf']!="not found") &  (train_df_1['qqdmr']!="not found")]
train_df_rfs = train_df_rf.shape
print("Rows where only item 1 could not be extracted {} ".format(*train_df_rfs))

train_df_mdaf =  train_df_1[ (train_df_1['mdaf']=="not found") & (train_df_1['risk_factors']!="not found") &  (train_df_1['qqdmr']!="not found") ]
train_df_mdafs = train_df_mdaf.shape
print("Rows where only item 7 could not be extracted  {} ".format(*train_df_mdafs))

train_df_qqdmr =  train_df_1[ (train_df_1['qqdmr']=="not found") & (train_df_1['mdaf']!="not found") & (train_df_1['risk_factors']!="not found") ]
train_df_qqdmrs = train_df_qqdmr.shape
print("Rows where only item 7a could not be extracted {} ".format(*train_df_qqdmrs))

train_df_all =  train_df_1[ (train_df_1['risk_factors']=="not found") & (train_df_1['mdaf']=="not found") &  (train_df_1['qqdmr']=="not found")]
train_df_all = train_df_all.shape
print("Rows where non of the item could be extracted  {} ".format(*train_df_all))

Rows where a parsing error occured 118 
Rows where only item 1 could not be extracted 155 
Rows where only item 7 could not be extracted  8 
Rows where only item 7a could not be extracted 12 
Rows where non of the item could be extracted  3810 


In [ ]:
#save df
#train_df_.to_csv(r"parsed_all.csv")